In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import os
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import SVR, LinearSVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from yellowbrick.cluster import KElbowVisualizer
from sklearn.preprocessing import StandardScaler



from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
desktop = os.path.join(os.path.join(os.environ['USERPROFILE']), 'Desktop') 

os.chdir(desktop)

In [ ]:
class DataInfo():
    def __init__(self, filename):
        import pandas as pd
        import numpy as np
        import os
        if filename[-4:] == ".xls" or filename[-5:] == ".xlsx":
            self.data = pd.read_excel(filename)
        elif filename[-4:] == ".csv":
            self.data = pd.read_csv(filename)

    def dfhead(self, rows=None):
        #veri tablosunun ilk 5 satırı
        if rows != None:
            return self.data.head(rows)
        return self.data.head()
               
    def null(self):
        null = self.data.isnull()
        return null
    
    def dfdescribe(self):
        describe = self.data.describe().T
        return describe
    
    def dfinfo(self):
        info = self.data.info()
        return info 
    
    def dfdropna(self):
        return self.data.dropna()
    
    def missing(self):
        missing = self.data.isnull().sum()
        return missing
    
    def countval(self, col):
        counts = self.data[col].value_counts()
        return counts
    

class Visualize():
    def __init__(self, data):
        import seaborn as sns
        import matplotlib.pyplot as plt
        self.data = data
    
    def hist(self):
        import seaborn as sns
        import matplotlib.pyplot as plt
        df = self.data
        numeric_cols = df.select_dtypes(include=["int64", "float64"])
        #fig, ax = plt.subplots(len(numeric_cols.columns), 1, figsize = (10, 30))
        #print(numeric_cols.iloc[:,0])
        for i in range (len(numeric_cols.columns)):
            plt.hist(numeric_cols.iloc[:,i], bins=40)
            #ax[i].hist(numeric_cols.iloc[:,i], bins=40)
            plt.title(numeric_cols.columns[i])
            plt.show()
            
    
    def boxplot(self):
        import seaborn as sns
        import matplotlib.pyplot as plt
        df = self.data
        numeric_cols = df.select_dtypes(include=["int64", "float64"])
        numeric_cols.boxplot(figsize=(20,15))
        
    
    def countplot(self):
        import seaborn as sns
        import matplotlib.pyplot as plt
        df = self.data
        categoric_cols = df.select_dtypes(exclude=["int64", "float64"])
        fig, ax = plt.subplots(len(categoric_cols.columns),1, figsize=(10,7))
        for i in range (len(categoric_cols.columns)):
            sns.countplot(x=categoric_cols.iloc[:,i],palette='pastel', ax = ax[i])
            
    def correlation(self):
        import seaborn as sns
        import matplotlib.pyplot as plt
        df = self.data
        fig,ax = plt.subplots(figsize=(30, 30))
        sns.heatmap(df.corr(), ax=ax, annot=True, 
            linewidths=0.05, fmt= '.2f',cmap="summer_r")
        plt.show()
        
    def jointplot(self):
        import seaborn as sns
        import matplotlib.pyplot as plt
        df = self.data
        numeric_cols = df.select_dtypes(include=["int64", "float64"])
        #fig, ax = plt.subplots(len(numeric_cols.columns),1, figsize=(10,7))
        for i in range (len(numeric_cols.columns)):
            for j in range(len(numeric_cols.columns)):
                if i == j:
                    continue
                sns.jointplot(x=numeric_cols.iloc[:,i],y=numeric_cols.iloc[:,j], data=df, kind="reg")
        #plt.show()
    


        
    

In [1]:
class Models():
    def __init__(self, data):
        self.data = data
    
    def split(self, targetcol, to_drop = []):
        df = self.data
        
        self.y = df[targetcol]
        self.X = df.drop(to_drop,axis=1)
        self.X = pd.DataFrame(self.X)
         
        X_train,X_test,y_train,y_test = train_test_split(self.X,self.y,test_size=0.30,random_state=42)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
    
    
    def LogReg(self):
        loj = sm.Logit(self.y_train,self.X_train)
        loj_model = loj.fit()
        summary = loj_model.summary()
    
        loj2 = LogisticRegression(solver = "liblinear")
        logreg = loj2.fit(self.X, self.y)
        intercept = logreg.intercept_
        coefs = logreg.coef_
    
        print (f"OLS: {summary}")
        print (f"SKLearn LogReg: {logreg}")
        print (f"LogReg Intercept: {intercept}")
        print (f"LogReg Coefficients: {coefs}")
        
        
        
        
    def LogRegPrediction(self):
    
        loj = LogisticRegression(solver = "liblinear") 
        lojmodel = loj.fit(self.X_train,self.y_train)
    
        y_pred = lojmodel.predict(self.X_test)
    
        accuracy = accuracy_score(self.y_test,y_pred)
        cm = confusion_matrix(self.y_test,y_pred)
        classreport = classification_report(self.y_test,y_pred)
    
        logit_roc_auc = roc_auc_score(self.y_train,lojmodel.predict(self.X_train))
        fpr, tpr, thresholds = roc_curve(self.y_train,lojmodel.predict_proba(self.X_train)[:,1])

        ROC = plt.figure(),
        plt.plot(fpr, tpr, label = "AUC (area = %0.2f)"% logit_roc_auc),
        plt.plot([0,1], [0,1], "r--"),
        plt.xlim([0.0, 1.0]),
        plt.ylim([0.0,1.05]),plt.xlabel("FP Oranı"),
        plt.ylabel("TP Oranı"),
        plt.title("ROC")
    
    
        cross_val_train = cross_val_score(lojmodel, self.X_train, self.y_train, cv = 10).mean()
        cross_val_test = cross_val_score(lojmodel, self.X_test, self.y_test, cv = 10).mean()

    
    
        print(f"Accuracy : {accuracy}")
        print("-------------------")
        print(f"Confusion Matrix: \n {cm}")
        print("-------------------")
        print(f"Classification report: \n {classreport}")
        print("-------------------")
        print(f"""Cross validation accuracy values: 
              training: {cross_val_train}
              test: {cross_val_test}""")
        print("-------------------")
        print(ROC)

    def pca(self):
        pca = PCA()
        self.X_reduced_train = pca.fit_transform(scale(self.X_train))
        cumulativesum = np.cumsum(np.round(pca.explained_variance_ratio_, decimals = 4)*100)[:]
        components = range(pca.n_components_)
    
        figure = plt.figure(figsize=(15,10)),
        plt.bar(components,pca.explained_variance_ratio_,color = "red"),
        plt.xlim(0,25),
        plt.xlabel("PCA Bilesenler"),
        plt.ylabel("Aciklanan Varyans %"),
        plt.xticks(components)
        plt.show()
    
        lm = LinearRegression()
        pca_model = lm.fit(self.X_reduced_train[:,:], self.y_train)  

        ols = sm.OLS(self.y_train,self.X_reduced_train[:,:])
        pca_model2  = ols.fit()
    
        y_pred = pca_model.predict(self.X_reduced_train[:,:])
        rmse1 = np.sqrt(mean_squared_error(self.y_train,y_pred))
        
        PCA_components = pd.DataFrame(self.X_reduced_train)
        PCAPlot = plt.scatter(PCA_components[0] , PCA_components[1], alpha = .1 , color='black'),
        plt.xlabel('PCA 1'),
        plt.ylabel('PCA 2');
        
        print(f"Bilesen sayisina gore aciklanabilirlik: \n {cumulativesum}")
        print(f"Açıklayıcılık grafiği: \n {figure}")
        print(f"Intercept: \n {pca_model.intercept_}")
        print(f"Coefs: \n {pca_model.coef_}")
        print(f"OLS tablosu : \n {pca_model2.summary().tables[0]}")
        print(f"PCA sonrası RMSE değeri: {rmse1}")
        print(PCAPlot)
     

    def nnclassifier(self):
        scaler = StandardScaler()
        X_train_scale = scaler.fit_transform(self.X_train)
        X_test_scale = scaler.fit_transform(self.X_test)

        mlp_regres = MLPClassifier().fit(X_train_scale,self.y_train) 
        y_pred = mlp_regres.predict(X_test_scale)
        accuracy = accuracy_score(self.y_test,y_pred)
        cm = confusion_matrix(self.y_test,y_pred)

        mlp_params = {"alpha":[0.1,0.01,0.02,0.005], 
              "hidden_layer_sizes":[(20,20),(100,50,150),(300,200,150)],
              'solver': ['adam', 'lbfgs']} 

        mlp_c = MLPClassifier()
        mlp_c_model = GridSearchCV(mlp_c,mlp_params,
                       cv = 10,
                       n_jobs = -1,
                       verbose = 2)

        mlp_tuned = MLPClassifier(alpha=list(mlp_c_model.best_params_.values())[0],
                                  hidden_layer_sizes=list(mlp_c_model.best_params_.values())[1],
                         solver=list(mlp_c_model.best_params_.values())[2]).fit(X_train_scale,y_train)

        y_pred = mlp_tuned.predict(X_test_scale)

        accuracy_tuned = accuracy_score(self.y_test,y_pred)
        cm_tuned = confusion_matrix(self.y_test,y_pred)


        print("Accuracy Before Tuning: ",accuracy)
        print("CM before tuning: \n",cm)
        print("Accuracy tuned:",accuracy_tuned)
        print("CM tuned: \n",cm_tuned)
        
        
        


    def nnregressor(self):
        scaler = StandardScaler()
        X_train_scale = scaler.fit_transform(self.X_train)
        X_test_scale =  scaler.fit_transform(self.X_test)
  
        mlp_reg = MLPRegressor()
        mlp_reg_model = mlp_reg.fit(X_train_scale,self.y_train) 
        y_pred = mlp_reg_model.predict(X_test_scale)
        rmse = np.sqrt(mean_squared_error(self.y_test,y_pred))

        mlp_params = {"alpha":[0.1,0.01,0.02,0.005], 
             "hidden_layer_sizes":[(20,20),(100,50,150),(300,200,150)],
              'solver': ['adam', 'lbfgs'],
              'learning_rate': ['constant','adaptive']} 

        mlp_cv_model = GridSearchCV(mlp_reg,mlp_params,cv = 10)
        mlp_cv_model = mlp_cv_model.fit(X_train_scale,self.y_train)
        mlp_tuned = MLPRegressor(
                         alpha=list(mlp_cv_model.best_params_.values())[0],
                        hidden_layer_sizes=list(mlp_cv_model.best_params_.values())[1],
                         learning_rate=list(mlp_cv_model.best_params_.values())[2],
                         solver=list(mlp_cv_model.best_params_.values())[3]).fit(X_train_scale,y_train)

        y_pred = mlp_tuned.predict(X_test_scale)

        rmse_tuned = np.sqrt(mean_squared_error(self.y_test,y_pred))

        print("RMSE before tuning: ",rmse)
        print("RMSE after tuning:",rmse_tuned)
        
        
    def dectreeregressor(self):
        
        cart = DecisionTreeRegressor()
        cart_model = cart.fit(self.X_train,self.y_train)

        y_pred = cart_model.predict(self.X_test)
        rmse1 = np.sqrt(mean_squared_error(self.y_test, y_pred))

        cart_grid = {"max_depth": range(1,10),
            "min_samples_split" : list(range(2,50))}

        cart_cv_model = GridSearchCV(cart_model, cart_grid, cv = 10, n_jobs=-1, verbose=2)

        cart_cv_model.fit(self.X_train, self.y_train)

        cart_tuned = DecisionTreeRegressor(max_leaf_nodes = list(cart_cv_model.best_params_.values())[0],
                                           min_samples_split = list(cart_cv_model.best_params_.values())[1])
        cart_tuned.fit(self.X_train, self.y_train)
        y_pred = cart_tuned.predict(self.X_test)
        rmse_tuned = np.sqrt(mean_squared_error(self.y_test, y_pred)) 


        print(f"RMSE (ilk değer): \n {rmse1}\n RMSE tuned: \n {rmse_tuned}")
        
        
        
    def dectreeclassifier(self):
        cart = DecisionTreeClassifier()
        cart_model = cart.fit(self.X_train, self.y_train)
        y_pred = cart_model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        cm = confusion_matrix(self.y_test,y_pred)
        
        print(f"Decision tree classifier accuracy: {accuracy}")
        print(f"Decision Tree Confusion Matrix: \n {cm}")

        cart_grid = {"max_depth": range(1,10),
            "min_samples_split" : list(range(2,50))}

        cart = DecisionTreeClassifier()
        cart_cv = GridSearchCV(cart,cart_grid, cv = 10,n_jobs=-1,verbose=2)
        cart_cv_model = cart_cv.fit(self.X_train, self.y_train)

        bos = []

        for i in cart_cv_model.best_params_:
              bos.append(cart_cv_model.best_params_[i])

        cart = DecisionTreeClassifier(max_depth = list(cart_cv_model.best_params_.values())[0],
                                      min_samples_split = list(cart_cv_model.best_params_.values())[1])
        cart_tuned = cart.fit(self.X_train, self.y_train)

        y_pred = cart_tuned.predict(self.X_test)
        accuracy_tuned = accuracy_score(self.y_test, y_pred)
        cm_tuned = confusion_matrix(self.y_test,y_pred)

        print("Tuned accuracy ve confusion matrix: ")
        print(accuracy_tuned)
        print(cm_tuned) 

    def rfclassifier(self):
        rf_model = RandomForestClassifier().fit(self.X_train, self.y_train)
        y_pred = rf_model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)  
        cm = confusion_matrix(self.y_test,y_pred)

        rf_params = {"max_depth": [2,5,8,10],
            "max_features": [2,5,8],
            "n_estimators": [10,500,1000],
            "min_samples_split": [2,5,10]}


        rf_model = RandomForestClassifier()
        rf_cv_model = GridSearchCV(rf_model, rf_params, cv = 10,
                           n_jobs = -1,
                           verbose = 2)

        rf_cv_model.fit(self.X_train,self.y_train)  

        rf_tuned = RandomForestClassifier(max_depth=list(rf_cv_model.best_params_.values())[0],
                                            max_features=list(rf_cv_model.best_params_.values())[1],
                                            min_samples_split=list(rf_cv_model.best_params_.values())[2],
                                            n_estimators=list(rf_cv_model.best_params_.values())[3])

        rf_tuned = rf_tuned.fit(self.X_train,self.y_train)
        y_pred = rf_tuned.predict(self.X_test)

        accuracy_tuned = accuracy_score(self.y_test,y_pred)
        cm_tuned = confusion_matrix(self.y_test,y_pred)


        Importance = pd.DataFrame({"Importance": rf_tuned.feature_importances_*100},
                         index = self.X_train.columns)
        importance = Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "yellow"),plt.xlabel("Değişken Önem Düzeyleri")

        print(f"En iyi parametre degerleri: {rf_cv_model.best_params_}")
        print("                                               ")
        print("Tune oncesi Accuracy ve Confusion Matrix degerleri: ",accuracy,"\n",cm)
        print("Tune sonrasi Accuracy ve Confusion Matrix degerleri: ",accuracy_tuned,"\n",cm_tuned)
        print("                                                ")
        print(importance)

    def rfregressor(self):
        rf_model = RandomForestRegressor()
        rf_model.fit(self.X_train, self.y_train)
        y_pred = rf_model.predict(self.X_test)
        rmse = np.sqrt(mean_squared_error(self.y_test, y_pred))

        rf_params = {'max_depth': list(range(1,10)),
            'max_features': [3,5,10,15],
            'n_estimators' : [100, 200, 500, 1000, 2000],
            'min_samples_leaf': [1, 2, 4],
            'min_samples_split': [2, 5, 10]}

        rf_model = RandomForestRegressor()

        rf_cv_model = GridSearchCV(rf_model, 
                           rf_params, 
                           cv = 10, 
                            n_jobs = -1,verbose = 2)

        rf_cv_model.fit(X_train, y_train)

        rf_tuned = RandomForestRegressor(max_depth  = list(rf_cv_model.best_params_.values())[0], 
                                 max_features = list(rf_cv_model.best_params_.values())[1], 
                                 min_samples_leaf = list(rf_cv_model.best_params_.values())[2],
                                 min_samples_split = list(rf_cv_model.best_params_.values())[3],
                                 n_estimators = list(rf_cv_model.best_params_.values())[4])

        rf_tuned.fit(self.X_train, self.y_train)
        y_pred = rf_tuned.predict(self.X_test)
        rmse_tuned = np.sqrt(mean_squared_error(self.y_test, y_pred))

        Importance = pd.DataFrame({"Importance": rf_tuned.feature_importances_*100},
                         index = self.X_train.columns)
        importance = Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "r"),plt.xlabel("Değişken Önem Düzeyleri")

        print("En iyi parametre degerleri: ",rf_cv_model.best_params_)
        print("                                               ")
        print("RMSE before tuning: ",rmse)
        print("RMSE tuned: ",rmse_tuned)
        print("                                                ")
        print(importance)

    def svmclass(self):
        
        svm_model = SVC(kernel="linear").fit(self.X_train, self.y_train)
        y_pred = svm_model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        class_report = classification_report(self.y_test, y_pred)

        svc_params= {"C": list(np.arange(1,10,1))}
        svc = SVC()
        
        svc_cv_model = GridSearchCV(svc, svc_params, 
                         cv = 10, 
                         n_jobs = -1,
                         verbose = 2)
        
        svc_tuned = SVC(C = list(svc_cv_model.best_params_.values())[0]).fit(self.X_train, self.y_train)
    
        y_pred = svc_tuned.predict(self.X_test)
        accuracy_tuned = accuracy_score(self.y_test, y_pred)
        class_report_tuned= classification_report(self.y_test,y_pred)
    
        print("Sınıflandırma raporu :\n",class_report_tuned)

    def svmregressor(self):
        svr = SVR("linear")
        svr_model = svr.fit(self.X_train, self.y_train)
        y_pred = svr_model.predict(self.X_test)
        rmse = np.sqrt(mean_squared_error(self.y_test, y_pred))
        
        svr_params = {"C": np.arange(1,10,1)}
        svr_cv_model = GridSearchCV(svr, svr_params, cv = 10,n_jobs=-1,verbose=2).fit(X_train,y_train)
        svr_tuned = SVR("linear", 
                C = pd.Series(svr_cv_model.best_params_)[0]).fit(X_train, y_train)

    
        y_pred = svr_tuned.predict(X_test)
        rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred))
    
        print("RMSE before tuning: \n",rmse)
        print("RMSE after tuning: ",rmse_tuned)
        

    def naivebayes(self):
        
        nb = GaussianNB()
        nb_model = nb.fit(self.X_train,self.y_train)
    
        nb_y_pred = nb_model.predict(self.X_test)
        nb_accuracy = accuracy_score(self.y_test,nb_y_pred)
    
        mnb = MultinomialNB()
        mnb_model = mnb.fit(self.X_train,self.y_train)
        
        mnb_y_pred = model.predict(self.X_test)
        mnb_accuracy = accuracy_score(self.y_test,mnb_y_pred)
    
        bnb = BernoulliNB()
        bernoulli_model = bnb.fit(self.X_train,self.y_train)
    
        bnb_y_pred = bernoulli_model.predict(self.X_test)
        bnb_accuracy = accuracy_score(self.y_test,bnb_y_pred)
    
        print("GaussianNB Accuracy: ", nb_accuracy)
        print("MultinomialNB accuracy: ", mnb_accuracy)
        print("BernoulliNB accuracy: ", bnb_accuracy)
        print("GaussianNB crossval: ", cross_val_score(nb_model, self.X_test, self.y_test, cv=10).mean())
        print("MultinomialNB crossval: ", cross_val_score(mnb_model, self.X_test, self.y_test, cv=10).mean())
        print("BernoulliNB crossval: ", cross_val_score(bnb_model, self.X_test, self.y_test, cv=10).mean())


    def lgbm_regressor(self):
        lgbm = LGBMRegressor()
        lgbm_model = lgbm.fit(self.X_train, self.y_train)
        
        y_pred = lgbm_model.predict(self.X_test)
        rmse = np.sqrt(mean_squared_error(self.y_test, y_pred))
        
        lgbm_grid = {
        'colsample_bytree': [0.4, 0.5,0.6,0.9,1],
        'learning_rate': [0.01, 0.1, 0.3,0.5,1],
        'n_estimators': [100,500,1000,2000],
        'max_depth': [1,2,3,4,5,6,7,8],
        "subsample": [0.1,0.3,0.5,0.8],
        "min_child_samples":[50,100,200]}
    
    
        lgbm = LGBMRegressor()
        lgbm_cv_model = GridSearchCV(lgbm, lgbm_grid, cv=10, n_jobs = -1, verbose = 2)
        lgbm_cv_model.fit(self.X_train, self.y_train)
    
        
        lgbm_tuned = LGBMRegressor(learning_rate = list(lgbm_cv_model.best_params_.values())[1], 
                           max_depth = list(lgbm_cv_model.best_params_.values())[2], 
                           n_estimators = list(lgbm_cv_model.best_params_.values())[4],
                          colsample_bytree = list(lgbm_cv_model.best_params_.values())[0],
                          subsample=list(lgbm_cv_model.best_params_.values())[5],
                          min_child_samples=list(lgbm_cv_model.best_params_.values())[3])

        lgbm_tuned = lgbm_tuned.fit(self.X_train,self.y_train)
    
        y_pred = lgbm_tuned.predict(self.X_test)
        rmse_tuned = np.sqrt(mean_squared_error(self.y_test, y_pred))
    
        print("Best parametreler: ",lgbm_cv_model.best_params_)
        print("RMSE before tuning: ",rmse)
        print("RMSE tuned: ",rmse_tuned)
        
    
    def lgbm_class(self):
        lgbm = LGBMClassifier()
        lgbm_model = lgbm.fit(self.X_train,self.y_train)

        y_pred = lgbm_model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        
        lgbm_params = {
        'n_estimators': [100, 500, 1000],
        'subsample': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5, 6],
        'learning_rate': [0.001,0.01,0.1,0.2,0.3],
        "min_child_samples": [20,50,75,100]}

        lgbm_cv_model = GridSearchCV(lgbm, lgbm_params, 
                             cv = 10, 
                             n_jobs = -1, 
                             verbose = 2)

        lgbm_cv_model.fit(self.X_train, self.y_train)

        lgbm = LGBMClassifier(learning_rate = list(lgbm_cv_model.best_params_.values())[0], 
                       max_depth = list(lgbm_cv_model.best_params_.values())[1],
                       subsample = list(lgbm_cv_model.best_params_.values())[4],
                       n_estimators = list(lgbm_cv_model.best_params_.values())[3],
                       min_child_samples = list(lgbm_cv_model.best_params_.values())[2])
    
        lgbm_tuned = lgbm.fit(self.X_train,self.y_train)

        y_pred = lgbm_tuned.predict(self.X_test)
        accuracy_tuned = accuracy_score(self.y_test, y_pred)
        class_report_tuned = classification_report(self.y_test,y_pred)
    
    
        print("Best parametreler: ",lgbm_cv_model.best_params_)
        print("Accuracy tuned: ",accuracy)
        print("Tuned sınıflandırma raporu: \n",class_report_tuned)
  
    def knnclassifier(self):
        knn = KNeighborsClassifier()
        knn_model = knn.fit(self.X_train, self.y_train)
        knn_model
        
        y_pred = knn_model.predict(self.X_test)

        accuracy = accuracy_score(self.y_test, y_pred)

        cr = classification_report(self.y_test, y_pred)
        
        knn_params = {"n_neighbors": np.arange(1,50),
             "metric": ["minkowski", "euclidean", "manhattan", "hamming"]}
        
        knn_cv = GridSearchCV(knn,knn_params, cv=10, n_jobs=-1, verbose=2)
        knn_cv.fit(self.X_train, self.y_train)
        
        knn = KNeighborsClassifier(n_neighbors=list(knn_cv.best_params_.values())[0], 
                                   metric = list(knn_cv.best_params_.values())[1])
        knn_tuned = knn.fit(self.X_train, self.y_train)
        
        y_pred = knn_model.predict(self.X_test)

        accuracy_tuned = accuracy_score(self.y_test, y_pred)

        cr_tuned = classification_report(self.y_test, y_pred)


        print("Best parametreler: \n",knn_cv.best_params_)
        print("KNN class accuracy: ",accuracy)
        print("KNN class classification report: \n",cr)
        print("KNN class accuracy tuned: ",accuracy_tuned)
        print("KNN class classification report tuned: \n",cr_tuned)
        
    def knnregressor(self):
        knnr = KNeighborsRegressor()
        knn_model = knnr.fit(self.X_train, self.y_train)
        knn_model
        
        y_pred = knn_model.predict(self.X_test)
        rmse=np.sqrt(mean_squared_error(self.y_test, y_pred))
        
        knn_params = {"n_neighbors": np.arange(1,30),
             "weights": ["uniform", "distance"],
             "metric": ["minkowski", "euclidean", "manhattan", "hamming"]}
        
        knn_cv = GridSearchCV(knnr, knn_params, cv=10, n_jobs=-1, verbose=2)
        knn_cv.fit(self.X_train, self.y_train)
        
        knnr = KNeighborsClassifier(n_neighbors=list(knn_cv.best_params_.values())[0], 
                                   metric = list(knn_cv.best_params_.values())[1])
        knn_tuned = knnr.fit(self.X_train, self.y_train)
        
        y_pred = knn_model.predict(self.X_test)

        rmse_tuned = np.sqrt(mean_squared_error(y_test, y_pred))


        print("Best parametreler: \n",knn_cv.best_params_)
        print("KNN class RMSE: ",rmse)
        print("KNN class RMSE tuned: ",rmse_tuned)
        

    def kelbow(self):
        df = self.data
        kmeans = KMeans()
        visualizer = KElbowVisualizer(kmeans, k=(1,15))
        visualizer.fit(df) 
        visualizer.poof()
        
    def cluster(self):
        df = self.X
        kmeans = KMeans(n_clusters = 4)
        k_fit = kmeans.fit(df)
        kumeler = k_fit.labels_
        plt.scatter(df.iloc[:,0], df.iloc[:,1], c = kumeler, s = 50, cmap = "viridis")
        merkezler = k_fit.cluster_centers_
        plt.scatter(merkezler[:,0], merkezler[:,1], c = "black", s = 200, alpha = 0.5)



In [ ]:
df = DataInfo("hmelq.csv") #please enter the filename here for classification

In [ ]:
df = df.dfdropna()

In [ ]:
vis = Visualize(df)

In [ ]:
models = Models(df)

In [ ]:
models.split("bad", ['bad', 'reason', "job"])

In [ ]:
models.LogRegPrediction()

In [ ]:
models.dectreeclassifier()

In [ ]:
models.knnclassifier()

In [ ]:
models.lgbm_class()

In [ ]:
models.naivebayes()

In [ ]:
models.nnclassifier()

In [ ]:
models.rfclassifier()

In [ ]:
models.svmclass()

In [ ]:
df = DataInfo("HW_Data_Set.xlsx") #please enter filename here for regression
df = df.dfdropna()
df = df.replace("?", np.mean)
vis = Visualize(df)
models = Models(df)

In [ ]:
models.split("50_target", ['ind_109', 'ind_420', 'ind_422'])

In [ ]:
models.dectreeregressor()

In [ ]:
models.knnregressor()

In [ ]:
models.lgbm_regressor()

In [ ]:
models.nnregressor()

In [ ]:
models.rfregressor()

In [ ]:
models.svmregressor()

In [ ]:
models.kelbow()

In [ ]:
models.cluster()